# Version 2.4: 중저가 예약 매장 요약문 생성

이 노트북은 **리뷰 데이터를 사용하지 않고**, 외부 정보만으로 중저가 예약 매장의 요약문을 생성합니다.

## 📌 핵심 특징

### 처리 모드
- 🔹 **단일 매장 모드**: 한 매장씩 상세하게 처리
- 🔹 **다중 매장 모드**: 여러 매장을 일괄 처리 (배치)

### 데이터 소스
- 📍 지역 정보 (양재천, 분당, 성수동 등)
- 🍽️ 시그니처 메뉴 및 조합
- 💰 가격대 및 가성비 정보
- 👥 타겟 고객층 (이탈리안 애호가, 여성 고객층, 젊은 층 등)
- 🎯 방문 용도 (데이트, 기념일, 가족 모임 등)
- 📺 화제성 (수요미식회, 흑백 요리사 등)

### 프롬프트 아키텍처
- **시스템/유저 프롬프트 분리 패턴** 사용
- **Few-shot Learning**: 10개의 고품질 예시로 학습
- **JSON 출력 강제**: 구조화된 결과 보장

### 출력 형식
```json
{
  "shop_seq": 1,
  "shop_name": "매장명",
  "title": "생면 파스타의 탄탄한 식감과 어란 파스타의 감칠맛 조합",
  "summaries": [
    "지역 + 컨셉 + 타겟 고객층 문장 (40-60자)",
    "시그니처 메뉴 상세 묘사 문장 (40-60자)",
    "공간 특징 + 방문 용도 문장 (40-60자)"
  ]
}
```

## 🚀 사용 워크플로우

### 단일 매장 모드
```
1. 섹션 1-3: 환경 설정 (최초 1회)
2. 섹션 4: MODE = "single" 설정 + 매장명 입력
3. 섹션 5: 웹 검색 정보 수집 및 붙여넣기
4. 섹션 6: 프롬프트 실행 → 요약문 생성
5. 섹션 7: 결과 검증 (선택)
6. 섹션 8: JSON 파일 저장 (선택)
```

### 다중 매장 모드
```
1. 섹션 1-3: 환경 설정 (최초 1회)
2. 섹션 4: MODE = "multi" 설정 + SHOP_LIST에 매장 정보 입력
3. 섹션 5: 건너뛰기
4. 섹션 6: 프롬프트 실행 → 여러 매장 일괄 생성
5. 섹션 7: 결과 검증 (선택)
6. 섹션 8: JSON + CSV 파일 저장 (선택)
```

## ⚠️ 섹션 1-3: 최초 1회만 실행 (수정 금지)

In [ ]:
# 섹션 1: Package 설치
!pip install google-cloud-aiplatform google-genai python-dotenv -q

In [ ]:
# 섹션 2: 라이브러리 Import
import os
import json
import pandas as pd
import vertexai
from google import genai
from google.genai import types
from dotenv import load_dotenv

In [ ]:
# 섹션 3: Vertex AI 초기화
# 환경 변수 로드
load_dotenv()

# GCP 프로젝트 설정
PROJECT_ID = "wad-dw"  # 실제 GCP 프로젝트 ID (프로젝트 이름 아님!)
LOCATION = "us-central1"

# Vertex AI 초기화
vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

print(f"✅ Vertex AI 초기화 완료: {PROJECT_ID} ({LOCATION})")

## ✅ 섹션 4: 매장 정보 입력 (수정 가능)

### 처리 모드 선택

#### 🔹 단일 매장 모드 (single)
- 한 매장씩 상세하게 처리
- 섹션 5에서 정보 수집
- 결과: JSON 파일 1개

#### 🔹 다중 매장 모드 (multi)
- 여러 매장을 일괄 처리
- SHOP_LIST에 모든 매장 정보 입력
- 섹션 5 건너뛰기
- 결과: JSON + CSV 파일

In [ ]:
# ========================================
# 매장 정보 입력
# ========================================
# 
# 두 가지 모드 지원:
# 1. 단일 매장 모드: SHOP_NAME에 하나의 매장명 입력
# 2. 다중 매장 모드: SHOPS 리스트 + COLLECTED_INFO 딕셔너리로 관리
# ========================================

# -----------------------------------------
# 모드 선택
# -----------------------------------------
MODE = "multi"  # "single" 또는 "multi"

# -----------------------------------------
# [단일 매장 모드] 단일 매장 정보
# -----------------------------------------
if MODE == "single":
    SHOP_SEQ = 1
    SHOP_NAME = "레스토랑 우오보"
    
    print(f"📍 단일 매장 모드")
    print(f"   - 매장명: {SHOP_NAME}")
    print(f"   - Shop Seq: {SHOP_SEQ}")

# -----------------------------------------
# [다중 매장 모드] 여러 매장 정보
# -----------------------------------------
elif MODE == "multi":
    # 1️⃣ 매장 기본 정보 (shop_seq, shop_name)
    SHOPS = [
        (1, "레스토랑 우오보"),
        (2, "퍼멘츠"),
        (3, "스시소라 정자점"),
    ]
    
    # 2️⃣ 매장별 수집된 정보 (매장명을 키로 사용)
    COLLECTED_INFO = {
        "매장1": "",
        "매장2": "",
        "매장3": "",
    }
    
    # 3️⃣ SHOP_LIST 자동 생성 (shop_seq, shop_name, collected_info 매핑)
    SHOP_LIST = [
        {
            "shop_seq": seq,
            "shop_name": name,
            "collected_info": COLLECTED_INFO.get(name, "[정보 없음 - COLLECTED_INFO에 추가 필요]")
        }
        for seq, name in SHOPS
    ]
    
    # 출력
    print(f"📍 다중 매장 모드")
    print(f"   - 총 매장 수: {len(SHOP_LIST)}개")
    for idx, shop in enumerate(SHOP_LIST, 1):
        info_length = len(shop['collected_info'].strip())
        status = "✅" if info_length > 100 else "⚠️"
        print(f"   {status} {idx}. {shop['shop_name']} (seq: {shop['shop_seq']}) - {info_length:,}자")
    print(f"\n⚠️ 섹션 5는 건너뛰고 바로 섹션 6을 실행하세요")

## ✅ 섹션 5: 웹 검색 및 정보 수집

### 정보 수집 방법

중저가 예약 매장은 다음 정보를 중심으로 수집합니다:

- 📍 **지역 정보** (양재천, 분당, 성수동 등 구체적 동네명)
- 🍽️ **시그니처 메뉴** (메뉴명, 조합, 식감, 맛)
- 💰 **가격대** (엔트리급, 합리적, 가성비)
- 👥 **타겟 고객층** (이탈리안 애호가, 여성층, 젊은 층)
- 📺 **화제성** (수요미식회, 흑백 요리사, SNS)
- 🎯 **방문 용도** (데이트, 기념일, 가족 모임)
- 🏛️ **공간 특징** (인테리어, 분위기, 룸 유무)

### 수집 프로세스

1. 매장명으로 웹 검색
2. 위 정보를 중심으로 수집
3. 아래 `collected_info` 변수에 붙여넣기
4. 섹션 6 실행하여 요약문 생성

In [ ]:
# 📝 수동 입력: 웹에서 수집한 정보를 아래에 입력하세요
# (단일 매장 모드에서만 사용)

if MODE == "single":
    collected_info = """
    [여기에 웹 검색 결과를 붙여넣으세요]

    예시:
    - 지역: 양재천 인근
    - 컨셉: 탄탄한 식감의 생면 파스타
    - 타겟: 이탈리안 미식 애호가
    - 시그니처 메뉴: 감자 뇨끼, 어란 파스타
    - 맛 특징: 파스타 간과 소스 밸런스 훌륭, 홍합살과 먹물 스파게티면의 감칠맛
    - 가격대: 중저가, 합리적
    - 공간: 깔끔하고 세련된 인테리어
    - 용도: 조용하고 격식 있는 데이트 장소
    """
    
    print("📥 웹 검색 정보 수집 완료 (단일 매장 모드)")
    print(f"   - 수집된 정보 길이: {len(collected_info):,} 글자")
    print(f"   - 추천 길이: 300~1,000 글자")
    if len(collected_info) < 200:
        print("   ⚠️ 정보가 너무 적습니다. 더 많은 정보를 수집해주세요.")

elif MODE == "multi":
    print("📥 다중 매장 모드")
    print("   ⚠️ 이 섹션은 건너뛰세요. 섹션 4에서 이미 정보를 입력했습니다.")
    print("   ➡️ 바로 섹션 6을 실행하세요.")

## ✅ 섹션 6: 프롬프트 작성 및 요약문 생성 (수정 가능)

### 프롬프트 구조 설계

이 노트북은 **시스템/유저 프롬프트 분리 패턴**을 사용합니다:

#### 📌 시스템 프롬프트 (System Prompt)
- **역할**: AI의 정체성, 규칙, 출력 형식 정의
- **특징**: 고정된 내용 (매장마다 동일)
- **내용**: 중저가 예약 매장 작성 원칙, 구조 규칙, 출력 형식

#### 📌 유저 프롬프트 (User Prompt)
- **역할**: 구체적인 작업 요청 및 데이터 제공
- **특징**: 매장마다 변경되는 내용
- **내용**: 매장 정보, 수집된 정보, Few-shot 예시

#### 🔗 프롬프트 결합
Google Gemini는 시스템/유저를 API 레벨에서 구분하지 않으므로, 전송 시 하나의 문자열로 결합합니다.

### 프롬프트 작성 규칙
- ❌ **f-string 사용 금지** (변수가 아직 정의되지 않아 에러)
- ✅ **일반 문자열 + `.format()` 사용**
- ✅ **JSON 예시에서 `{{` `}}` 사용** (이스케이프)

In [ ]:
# ========================================
# 프롬프트 정의
# ========================================
# 
# 프롬프트를 시스템/유저로 분리하는 이유:
# 1. 역할 분리: AI의 정체성(system) vs 구체적 작업(user)
# 2. 재사용성: system은 고정, user만 매번 변경
# 3. 관리 용이성: 규칙과 데이터를 명확히 구분
# 
# Google Gemini는 시스템/유저를 API 레벨에서 구분하지 않으므로,
# 전송 시에는 하나의 문자열로 결합합니다.
# ========================================

# -----------------------------------------
# [시스템 프롬프트] AI의 정체성과 규칙
# -----------------------------------------
system_prompt = """
당신은 중저가 예약 매장 전문 작가입니다.
리뷰 데이터 없이 외부 정보만으로 고객이 실용적으로 활용할 수 있는 매장 요약문을 작성합니다.

## 작성 원칙
1. **실용성**: 누가, 언제, 왜 가야 하는지 명확히 전달
2. **구체성**: 메뉴명, 지역명, 가격대를 구체적으로 명시
3. **직관성**: 식감과 맛을 직관적으로 표현 (과장 금지)
4. **친근함**: 격식보다 친근하고 솔직한 톤
5. **가이드성**: 방문 용도를 명확히 제시 (데이트, 기념일 등)
6. **독창성**: 예시를 그대로 복사하지 말고, 톤앤매너만 참고하여 해당 매장만의 고유한 특징을 작성

## 요약 문장 구조
- **1번 문장**: 지역 + 핵심 컨셉 + 타겟 고객층 + 평가/가격 밸류
- **2번 문장**: 시그니처 메뉴 2개 + 식감/맛/조합 상세 묘사
- **3번 문장**: 공간 특징 + 구체적 방문 용도 (데이트, 기념일, 가족 모임 등)

## 제목 작성 규칙
- 형식: "[시그니처 메뉴명]의 [식감/맛 형용사]와 [특징 요소] 조합"
- 예시: "생면 파스타의 탄탄한 식감과 어란 파스타의 감칠맛 조합"
- 길이: 20-35자
- 메뉴명 필수 포함

## 중저가 예약 매장 특화 가이드
- ✅ 지역명 명시 (양재천, 분당, 성수동 등)
- ✅ 타겟 고객층 명시 (이탈리안 애호가, 여성층, 젊은 층)
- ✅ 가격 언급 (엔트리급, 합리적, 가성비)
- ✅ 화제성 강조 (수요미식회, 흑백 요리사, SNS)
- ✅ 메뉴 조합 상세 (라즈베리 잼 + 매쉬포테이토)
- ✅ 식감 직관적 표현 (촉촉한, 사르르 녹는, 탄탄한)
- ✅ 방문 용도 명확히 (데이트 장소로, 기념일 식사로)
- ❌ 철학/기법 중심 표현 금지
- ❌ 과도한 전문 용어 사용 금지

## 출력 형식
반드시 유효한 JSON 형식으로만 출력하세요. 다른 설명이나 주석은 포함하지 마세요.
{{
  "shop_seq": 숫자,
  "shop_name": "매장명",
  "title": "제목",
  "summaries": ["문장1", "문장2", "문장3"]
}}
"""

# -----------------------------------------
# [유저 프롬프트 템플릿] 구체적 작업 내용
# -----------------------------------------
user_prompt_template = """
[매장 정보]
매장명: {shop_name}
Shop Seq: {shop_seq}

[수집된 정보]
{collected_info}

[참고 예시 - 아래 예시들의 톤앤매너와 문체만 참고하세요. 내용은 절대 복사하지 마세요.]

**중요**: 아래 예시는 작성 스타일을 보여주기 위한 참고용입니다. 예시의 내용을 그대로 사용하지 말고, 
수집된 정보를 바탕으로 해당 매장만의 고유한 특징을 작성하세요.

---

**예시 1: 레스토랑 우오보 (양재천)**
{{
  "title": "생면 파스타의 탄탄한 식감과 어란 파스타의 감칠맛 조합",
  "summaries": [
    "양재천 인근에서 탄탄한 식감의 생면 파스타를 내세워 이탈리안 미식 애호가들 사이에서 높은 평가를 받습니다. 파스타의 간과 소스 밸런스가 훌륭하다는 것이 공통된 의견입니다.",
    "시그니처인 감자 뇨끼는 완벽한 질감으로 유명하며, 어란 파스타는 홍합살과 먹물 스파게티면이 어우러져 감칠맛이 폭발하는 조합으로 높은 추천을 받습니다.",
    "실내는 깔끔하고 세련된 인테리어로 설계되어 있으며, 조용하고 격식 있는 데이트 장소로 꾸준히 화제성을 유지하는 분위기를 제공합니다."
  ]
}}

**예시 2: 퍼멘츠**
{{
  "title": "비건 다이닝의 완성도를 보여주는 컬리플라워 비건 치킨",
  "summaries": [
    "비건(Vegan)과 발효라는 트렌드를 선도하며, 채식 미식가들뿐 아니라 육식파 사이에서도 높은 완성도와 맛을 인정받아 압도적인 화제성을 구축했습니다.",
    "시그니처인 토마토 후무스와 구운 대파 후무스는 사워도우와 결합 시 중독적인 풍미를 선사합니다. 컬리플라워 비건 치킨은 훌륭한 식감으로 높은 추천을 받습니다.",
    "외관과 대비되는 아늑하고 이국적인 실내 분위기는 비건 다이닝에 대한 선입견을 깨고 편안한 식사를 유도합니다."
  ]
}}

**예시 3: 스시소라 정자점 (분당)**
{{
  "title": "금태 솥밥과 함께 코스를 완성하는 우니 크림소스 호타테",
  "summaries": [
    "엔트리급 가격대에서 구현하는 최대한의 퀄리티로 분당 지역에서 높은 가성비의 미들급 스시야로 평가받습니다.",
    "셰프가 쥐어주는 '우니 크림소스가 올라간 가리비'는 입안에서 사르르 녹는 식감을 자랑하며, 기름지고 구수한 맛이 일품인 금태 솥밥이 코스를 풍족하게 완성합니다.",
    "약간 강한 듯한 샤리가 네타와의 조화를 이루는 것이 특징입니다. 외관 및 내부가 모던하고 깔끔한 분위기로 꾸며져 있어 쾌적하고 차분하게 스시 오마카세를 경험하기에 적합합니다."
  ]
}}

**예시 4: 미트컬쳐 (흑백 요리사)**
{{
  "title": "웨디시 미트볼과 라즈베리 잼의 독특한 단짠 조합",
  "summaries": [
    "육류 메뉴에 특화되어 있으며, 흑백 요리사 셰프의 섬세한 정성이 느껴지며 독특한 맛을 제공한다는 평가를 받습니다.",
    "시그니처인 촉촉한 스웨디시 미트볼은 라즈베리 잼, 오이, 매쉬 포테이토와 함께 단짠의 조화를 이루는 조합으로 가장 유명합니다. 왕가자미 요리는 깊은 바다의 풍미를 선사합니다.",
    "트렌디하면서도 조용하고 운치 있는 내부 분위기 덕분에 캐주얼하면서도 예쁜 데이트 장소로 높은 평가를 받습니다."
  ]
}}

**예시 5: 그리노 성수**
{{
  "title": "옥수수 먹물 뇨끼와 화이트 라구 파스타의 깊은 풍미",
  "summaries": [
    "성수동 특유의 아기자기하고 감각적인 분위기를 갖춘 브런치 및 파스타 맛집으로, 크리미한 소스가 깊고 진한 풍미를 선사하여 여성 고객층 사이에서 높은 화제성을 보유하고 있습니다.",
    "시그니처 메뉴인 옥수수 먹물 뇨끼는 크리미한 소스에 버터향이 더해져 풍부한 맛을 구현합니다. 화이트 라구 파스타는 미트가 듬뿍 들어있고 트러플 향이 나는 조합으로 미식적 만족도를 높입니다.",
    "조도가 낮고 이탈리안 감성이 느껴지는 예쁜 인테리어는 연말 모임이나 데이트 장소로 제격이라는 평가를 받습니다."
  ]
}}

**예시 6: 밴건디스테이크하우스**
{{
  "title": "안심 부위의 부드러움과 시금치/알리고 매쉬포테이토 조합",
  "summaries": [
    "티본/포터하우스 스테이크를 합리적인 가격대에 제공하여 하이엔드 스테이크하우스의 훌륭한 대안으로 대중적 평가를 받습니다.",
    "안심 부위가 이빨 없이도 먹을 수 있을 정도의 부드러움을 자랑하며, 로메인 샐러드가 입맛을 돋우는 시그니처 조합입니다. 사이드 메뉴인 향이 강한 치즈 시금치나 알리고 질감의 매쉬 포테이토가 스테이크의 풍미를 돕습니다.",
    "가격대가 상당하지만 기념일 식사 장소로 손색없는 고급스럽고 격식 있는 분위기를 연출합니다."
  ]
}}

**예시 7: 몽중헌 청담점 (수요미식회)**
{{
  "title": "중국 현지 명인의 솜씨가 돋보이는 구채교와 하교 딤섬",
  "summaries": [
    "중국 현지 명인이 만드는 딤섬을 주력으로 내세워 정통 딤섬의 깊은 맛을 구현한다는 평가를 받습니다. 수요미식회에서도 소개될 만큼 높은 화제성을 보유했습니다.",
    "시그니처 딤섬은 구채교(부추 딤섬)와 하교(새우 딤섬)이며, 요리로는 소스 간이 적당한 가지 덮밥이 높은 추천을 받습니다.",
    "실내는 중국 선화(仙話) 속 신선의 세상을 모티브로 잔잔한 조명과 넓은 테이블, 다수의 룸 공간을 갖추어 가족 모임, 비즈니스 식사 장소로 적합한 고급스럽고 조용한 분위기를 제공합니다."
  ]
}}

**예시 8: 더기와 합정점**
{{
  "title": "육즙 보쌈과 비빔장의 조화, 보쌈 칼비빔면 추천 조합",
  "summaries": [
    "한식 주점으로서 익숙한 한식 메뉴를 트렌디하고 세련되게 재해석하여 젊은 층 사이에서 높은 화제성을 얻고 있습니다.",
    "시그니처 메뉴인 보쌈 칼비빔면은 육즙을 머금은 보쌈과 비빔장의 조화가 훌륭하며, 깻잎과 파 고명이 느끼함을 잡아주는 조합으로 유명합니다. 기본 안주로 제공되는 묵국수와 육전의 맛이 훌륭하여 술 페어링에 최적화되었다는 평가를 받습니다.",
    "기와집 무드의 외관과 네온사인이 어우러져 독특하고 아늑한 분위기를 자아내며, 단독 룸이 있어 프라이빗한 모임에 적합합니다."
  ]
}}

**예시 9: DOTZ (한남동)**
{{
  "title": "한남동에서 즐기는 큼지막한 카츠산도와 렌치 소스의 조합",
  "summaries": [
    "한남동에서 브런치와 아시아 퓨전 요리를 선보이는 매장으로, 다국적인 요리의 창의적인 구성과 깔끔한 플레이팅에 대한 대중적 평가가 높습니다.",
    "시그니처인 큼지막한 카츠산도가 거의 모든 테이블에서 주문될 정도로 인기가 높으며, 렌치 소스를 곁들여 먹는 조합이 훌륭하다는 평가를 받습니다. 라구 파스타 역시 꾸준히 추천되는 메뉴입니다.",
    "전반적으로 깔끔하고 조용한 분위기를 유지하며, 통창 근처 좌석은 날씨가 좋을 때 특히 아름다운 뷰를 선사합니다. 커플 방문이나 편안한 식사를 원하는 이들에게 적합합니다."
  ]
}}

**예시 10: 야키토리 해공**
{{
  "title": "짚불 와라야키 응용으로 강렬한 풍미를 선사하는 곰장어 꼬치구이",
  "summaries": [
    "야키토리 전문점으로, 특제 부산 곰장어 구이라는 독특한 시그니처 메뉴로 미식가들 사이에서 높은 화제성을 확보했습니다.",
    "곰장어 꼬치구이에 짚불에 굽는 '와라야키' 방식을 응용하여 강렬한 향과 탱글탱글한 식감을 구현합니다. 곰장어 꼬치구이는 입에 꽉 차는 크기와 풍성한 육즙이 특징입니다.",
    "실내는 세련된 디자인이 돋보이는 조용한 분위기로, 정통 야키토리와 부산 특유의 해산물 메뉴를 곁들인 이색적인 미식 경험을 제공합니다."
  ]
}}

---

**다시 한 번 강조**: 위 예시는 작성 스타일(톤, 문체, 구조)을 참고하기 위한 것입니다.
예시의 구체적인 내용(메뉴명, 지역명, 특징 등)을 그대로 사용하지 말고,
[수집된 정보]를 바탕으로 [{shop_name}] 매장만의 고유한 특징을 작성하세요.
"""

print("✅ 프롬프트 템플릿 작성 완료")
print("   - 시스템 프롬프트: 중저가 예약 매장 작성 원칙")
print("   - 유저 프롬프트: 구체적 작업 내용")
print("   - Few-shot 예시: 10개 (톤앤매너 참고용)")

In [ ]:
# ========================================
# 모델 설정 및 LLM 호출
# ========================================

# -----------------------------------------
# 모델 파라미터 설정
# -----------------------------------------
MODEL_NAME = "gemini-2.5-pro"  # 높은 품질
TEMPERATURE = 0.5  # 창의성과 일관성의 균형
MAX_OUTPUT_TOKENS = 4096  # JSON 응답을 위한 충분한 토큰

generation_config = types.GenerateContentConfig(
    temperature=TEMPERATURE,
    max_output_tokens=MAX_OUTPUT_TOKENS,
    response_mime_type="application/json"  # JSON 출력 강제
)

print(f"🤖 모델: {MODEL_NAME}")
print(f"   - Temperature: {TEMPERATURE}")
print(f"   - Max Output Tokens: {MAX_OUTPUT_TOKENS:,}\n")

# -----------------------------------------
# 요약문 생성 함수
# -----------------------------------------
def generate_summary(shop_seq, shop_name, collected_info):
    """단일 매장에 대한 요약문 생성"""
    
    # 유저 프롬프트 생성
    user_prompt = user_prompt_template.format(
        shop_seq=shop_seq,
        shop_name=shop_name,
        collected_info=collected_info
    )
    
    # 프롬프트 결합
    combined_prompt = f"{system_prompt}\n\n{user_prompt}"
    
    try:
        # LLM 호출
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=combined_prompt,
            config=generation_config
        )
        
        # JSON 파싱
        result_text = response.text
        result_json = json.loads(result_text)
        
        # 토큰 사용량 추출
        usage_metadata = response.usage_metadata if hasattr(response, 'usage_metadata') else None
        token_info = {}
        if usage_metadata:
            token_info = {
                "input_tokens": usage_metadata.prompt_token_count,
                "output_tokens": usage_metadata.candidates_token_count,
                "total_tokens": usage_metadata.total_token_count
            }
        
        return {
            "status": "success",
            "data": result_json,
            "tokens": token_info
        }
        
    except json.JSONDecodeError as e:
        return {
            "status": "error",
            "error_type": "JSONDecodeError",
            "message": str(e),
            "response": response.text if 'response' in locals() else None,
            "tokens": {}
        }
        
    except Exception as e:
        return {
            "status": "error",
            "error_type": type(e).__name__,
            "message": str(e),
            "response": response.text if 'response' in locals() else None,
            "tokens": {}
        }

# -----------------------------------------
# 단일/다중 모드 처리
# -----------------------------------------
all_results = []
total_input_tokens = 0
total_output_tokens = 0
total_tokens = 0

if MODE == "single":
    # 단일 매장 모드
    print(f"📝 [{SHOP_NAME}] 요약문 생성 중...\n")
    
    result = generate_summary(SHOP_SEQ, SHOP_NAME, collected_info)
    
    if result["status"] == "success":
        result_json = result["data"]
        all_results.append(result_json)
        
        # 토큰 사용량 출력
        tokens = result.get("tokens", {})
        if tokens:
            print(f"📊 토큰 사용량:")
            print(f"   - Input:  {tokens.get('input_tokens', 0):,} tokens")
            print(f"   - Output: {tokens.get('output_tokens', 0):,} tokens")
            print(f"   - Total:  {tokens.get('total_tokens', 0):,} tokens\n")
        
        print("✅ 요약문 생성 완료\n")
        print("=" * 80)
        print(json.dumps(result_json, ensure_ascii=False, indent=2))
        print("=" * 80)
    else:
        print(f"❌ 에러 발생: {result['error_type']}")
        print(f"   메시지: {result['message']}")
        if result.get('response'):
            print(f"   응답: {result['response'][:200]}...")

elif MODE == "multi":
    # 다중 매장 모드
    print(f"📝 {len(SHOP_LIST)}개 매장 요약문 생성 시작\n")
    print("=" * 80)
    
    for idx, shop in enumerate(SHOP_LIST, 1):
        shop_seq = shop["shop_seq"]
        shop_name = shop["shop_name"]
        collected_info = shop["collected_info"]
        
        print(f"\n[{idx}/{len(SHOP_LIST)}] {shop_name} 처리 중...")
        
        result = generate_summary(shop_seq, shop_name, collected_info)
        
        if result["status"] == "success":
            result_json = result["data"]
            all_results.append(result_json)
            
            # 토큰 사용량 출력 및 누적
            tokens = result.get("tokens", {})
            if tokens:
                input_tokens = tokens.get('input_tokens', 0)
                output_tokens = tokens.get('output_tokens', 0)
                total_tokens_shop = tokens.get('total_tokens', 0)
                
                total_input_tokens += input_tokens
                total_output_tokens += output_tokens
                total_tokens += total_tokens_shop
                
                print(f"   ✅ 성공 | Input: {input_tokens:,} | Output: {output_tokens:,} | Total: {total_tokens_shop:,} tokens")
            else:
                print(f"   ✅ 성공")
        else:
            print(f"   ❌ 실패: {result['error_type']} - {result['message']}")
            # 실패한 경우에도 기록
            all_results.append({
                "shop_seq": shop_seq,
                "shop_name": shop_name,
                "status": "error",
                "error": result['message']
            })
    
    print("\n" + "=" * 80)
    print(f"\n✅ 전체 완료: {len(all_results)}개 매장 처리")
    
    # 성공/실패 개수
    success_count = sum(1 for r in all_results if r.get("status") != "error")
    error_count = len(all_results) - success_count
    print(f"   - 성공: {success_count}개")
    if error_count > 0:
        print(f"   - 실패: {error_count}개")
    
    # 총 토큰 사용량 출력
    if total_tokens > 0:
        print(f"\n📊 총 토큰 사용량:")
        print(f"   - Total Input:  {total_input_tokens:,} tokens")
        print(f"   - Total Output: {total_output_tokens:,} tokens")
        print(f"   - Total:        {total_tokens:,} tokens")
    
    # -----------------------------------------
    # 전체 결과 상세 출력
    # -----------------------------------------
    print("\n" + "=" * 80)
    print("📋 전체 결과 상세보기")
    print("=" * 80)
    
    for idx, result in enumerate(all_results, 1):
        print(f"\n{'─' * 80}")
        print(f"[{idx}] {result.get('shop_name')}")
        print(f"{'─' * 80}")
        
        if result.get("status") != "error":
            # 성공한 경우 - 전체 내용 출력
            print(f"\n📌 제목:")
            print(f"   {result.get('title')}")
            
            print(f"\n📝 요약문:")
            summaries = result.get('summaries', [])
            for i, summary in enumerate(summaries, 1):
                print(f"   {i}. {summary}")
            
            print(f"\n✅ 상태: 성공")
        else:
            # 실패한 경우
            print(f"\n❌ 상태: 에러")
            print(f"   오류 내용: {result.get('error', 'N/A')}")
    
    print("\n" + "=" * 80)

## ✅ 섹션 7: 후처리 검증 (선택 사항)

### 검증 항목

이 섹션은 생성된 요약문의 품질을 **코드 기반**으로 검증합니다 (LLM 호출 없음):

#### 📋 기본 구조 검증
- 필수 키 존재 확인 (shop_seq, shop_name, title, summaries)
- 요약문 개수 (정확히 3개)
- 문장 길이 (40-60자 권장, 30자 미만/100자 초과 경고)
- 제목 길이 (20-35자 권장)

#### 🚫 품질 검증
- 예시 복사 의심 (예시와 90% 이상 유사)
- 과장된 표현 금지 (최고의, 완벽한, 최상의 등)
- 문장 간 중복 내용 (3개 문장이 너무 유사)
- 특수문자/이모지 체크

#### 💡 검증 결과
- ✅ 통과: 모든 규칙 준수
- ⚠️ 경고: 일부 권장사항 위반 (사용 가능하지만 검토 권장)
- ❌ 실패: 필수 규칙 위반 (수정 필요)

In [ ]:
# ========================================
# 강력한 후처리 검증 함수
# ========================================

import difflib
import re

# -----------------------------------------
# 예시 문장 리스트 (복사 의심 체크용)
# -----------------------------------------
EXAMPLE_SENTENCES = [
    "양재천 인근에서 탄탄한 식감의 생면 파스타를 내세워 이탈리안 미식 애호가들 사이에서 높은 평가를 받습니다. 파스타의 간과 소스 밸런스가 훌륭하다는 것이 공통된 의견입니다.",
    "시그니처인 감자 뇨끼는 완벽한 질감으로 유명하며, 어란 파스타는 홍합살과 먹물 스파게티면이 어우러져 감칠맛이 폭발하는 조합으로 높은 추천을 받습니다.",
    "실내는 깔끔하고 세련된 인테리어로 설계되어 있으며, 조용하고 격식 있는 데이트 장소로 꾸준히 화제성을 유지하는 분위기를 제공합니다.",
    "비건(Vegan)과 발효라는 트렌드를 선도하며, 채식 미식가들뿐 아니라 육식파 사이에서도 높은 완성도와 맛을 인정받아 압도적인 화제성을 구축했습니다.",
    "시그니처인 촉촉한 스웨디시 미트볼은 라즈베리 잼, 오이, 매쉬 포테이토와 함께 단짠의 조화를 이루는 조합으로 가장 유명합니다.",
    "성수동 특유의 아기자기하고 감각적인 분위기를 갖춘 브런치 및 파스타 맛집으로, 크리미한 소스가 깊고 진한 풍미를 선사하여 여성 고객층 사이에서 높은 화제성을 보유하고 있습니다.",
]

# -----------------------------------------
# 금지 키워드 리스트 (과장된 표현)
# -----------------------------------------
BANNED_KEYWORDS = [
    "최고의", "최상의", "완벽한", "최고급의", "세계 최고",
    "압도적인", "독보적인", "타의 추종을 불허하는",
    "놀라운", "경이로운", "환상적인", "굉장한"
]

def calculate_similarity(text1, text2):
    """두 문자열 간 유사도 계산 (0.0 ~ 1.0)"""
    return difflib.SequenceMatcher(None, text1, text2).ratio()

def check_example_similarity(sentence):
    """예시 문장과의 유사도 체크"""
    max_similarity = 0.0
    most_similar_example = None
    
    for example in EXAMPLE_SENTENCES:
        similarity = calculate_similarity(sentence, example)
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_example = example
    
    return max_similarity, most_similar_example

def check_banned_keywords(text):
    """금지 키워드 체크"""
    found_keywords = []
    for keyword in BANNED_KEYWORDS:
        if keyword in text:
            found_keywords.append(keyword)
    return found_keywords

def check_special_chars(text):
    """특수문자/이모지 체크"""
    # 이모지 패턴
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # 이모티콘
        "\U0001F300-\U0001F5FF"  # 기호 & 픽토그램
        "\U0001F680-\U0001F6FF"  # 교통 & 지도
        "\U0001F1E0-\U0001F1FF"  # 국기
        "]+", 
        flags=re.UNICODE
    )
    
    emojis = emoji_pattern.findall(text)
    return emojis

def validate_summary(result, strict=False):
    """
    요약문 검증 함수
    
    Args:
        result: 검증할 결과 dict
        strict: True이면 경고도 실패로 간주
    
    Returns:
        dict: {
            "passed": bool,
            "errors": list,
            "warnings": list
        }
    """
    errors = []
    warnings = []
    
    # -----------------------------------------
    # 1. 필수 키 존재 확인
    # -----------------------------------------
    required_keys = ["shop_seq", "shop_name", "title", "summaries"]
    for key in required_keys:
        if key not in result:
            errors.append(f"❌ 필수 키 누락: {key}")
    
    if errors:  # 필수 키가 없으면 더 이상 검증 불가
        return {
            "passed": False,
            "errors": errors,
            "warnings": warnings
        }
    
    # -----------------------------------------
    # 2. 요약문 개수 확인
    # -----------------------------------------
    summaries = result.get("summaries", [])
    if len(summaries) != 3:
        errors.append(f"❌ 요약문 개수 오류: {len(summaries)}개 (3개 필요)")
        return {
            "passed": False,
            "errors": errors,
            "warnings": warnings
        }
    
    # -----------------------------------------
    # 3. 문장 길이 확인
    # -----------------------------------------
    for i, sentence in enumerate(summaries, 1):
        length = len(sentence)
        if length < 30:
            warnings.append(f"⚠️ {i}번 문장이 너무 짧음: {length}자 (권장 40-60자)")
        elif length > 100:
            warnings.append(f"⚠️ {i}번 문장이 너무 김: {length}자 (권장 40-60자)")
    
    # -----------------------------------------
    # 4. 제목 길이 확인
    # -----------------------------------------
    title = result.get("title", "")
    title_length = len(title)
    if title_length < 15:
        warnings.append(f"⚠️ 제목이 너무 짧음: {title_length}자 (권장 20-35자)")
    elif title_length > 45:
        warnings.append(f"⚠️ 제목이 너무 김: {title_length}자 (권장 20-35자)")
    
    # -----------------------------------------
    # 5. 예시 복사 의심 체크
    # -----------------------------------------
    for i, sentence in enumerate(summaries, 1):
        similarity, similar_example = check_example_similarity(sentence)
        if similarity >= 0.9:
            errors.append(f"❌ {i}번 문장이 예시를 복사한 것으로 의심됨 (유사도 {similarity:.1%})")
            errors.append(f"   원본: {similar_example[:50]}...")
        elif similarity >= 0.7:
            warnings.append(f"⚠️ {i}번 문장이 예시와 유사함 (유사도 {similarity:.1%})")
    
    # -----------------------------------------
    # 6. 금지 키워드 체크
    # -----------------------------------------
    all_text = title + " ".join(summaries)
    banned_found = check_banned_keywords(all_text)
    if banned_found:
        warnings.append(f"⚠️ 과장된 표현 발견: {', '.join(banned_found)}")
    
    # -----------------------------------------
    # 7. 특수문자/이모지 체크
    # -----------------------------------------
    emojis = check_special_chars(all_text)
    if emojis:
        warnings.append(f"⚠️ 이모지 발견: {', '.join(emojis)}")
    
    # -----------------------------------------
    # 8. 문장 간 중복 체크 (3개 문장이 너무 유사)
    # -----------------------------------------
    for i in range(len(summaries)):
        for j in range(i+1, len(summaries)):
            similarity = calculate_similarity(summaries[i], summaries[j])
            if similarity >= 0.7:
                warnings.append(f"⚠️ {i+1}번과 {j+1}번 문장이 너무 유사함 (유사도 {similarity:.1%})")
    
    # -----------------------------------------
    # 결과 판정
    # -----------------------------------------
    if strict:
        passed = len(errors) == 0 and len(warnings) == 0
    else:
        passed = len(errors) == 0
    
    return {
        "passed": passed,
        "errors": errors,
        "warnings": warnings
    }

# -----------------------------------------
# 전체 결과 검증 실행
# -----------------------------------------
if len(all_results) == 0:
    print("⚠️ 검증할 결과가 없습니다.")
    print("   먼저 섹션 6을 실행하여 요약문을 생성하세요.")
else:
    print("🔍 요약문 검증 시작\n")
    print("=" * 80)
    
    total_passed = 0
    total_warnings = 0
    total_failed = 0
    
    validation_results = []
    
    for idx, result in enumerate(all_results, 1):
        # 에러 상태인 결과는 스킵
        if result.get("status") == "error":
            print(f"\n[{idx}] {result.get('shop_name')}: 생성 실패로 검증 스킵")
            continue
        
        shop_name = result.get("shop_name", "Unknown")
        print(f"\n[{idx}] {shop_name}")
        print("─" * 80)
        
        # 검증 실행
        validation = validate_summary(result, strict=False)
        validation_results.append({
            "shop_name": shop_name,
            "validation": validation
        })
        
        errors = validation["errors"]
        warnings = validation["warnings"]
        passed = validation["passed"]
        
        if passed and len(warnings) == 0:
            print("✅ 검증 통과: 모든 규칙을 준수합니다")
            total_passed += 1
        elif passed:
            print(f"✅ 검증 통과 (경고 {len(warnings)}개)")
            total_passed += 1
            total_warnings += 1
            for warning in warnings:
                print(f"   {warning}")
        else:
            print(f"❌ 검증 실패 (오류 {len(errors)}개, 경고 {len(warnings)}개)")
            total_failed += 1
            for error in errors:
                print(f"   {error}")
            for warning in warnings:
                print(f"   {warning}")
    
    # -----------------------------------------
    # 전체 검증 요약
    # -----------------------------------------
    print("\n" + "=" * 80)
    print("📊 검증 요약")
    print("=" * 80)
    print(f"✅ 통과: {total_passed}개")
    if total_warnings > 0:
        print(f"⚠️ 경고 있음: {total_warnings}개")
    if total_failed > 0:
        print(f"❌ 실패: {total_failed}개")
    
    success_rate = (total_passed / len([r for r in all_results if r.get("status") != "error"])) * 100
    print(f"\n성공률: {success_rate:.1f}%")

## ✅ 섹션 8: 결과 저장 (선택 사항)

### 저장 형식

#### 단일 매장 모드
- JSON 파일 1개 (전체 구조 포함)
- 파일명: `summary_{매장명}_{타임스탬프}.json`

#### 다중 매장 모드
- JSON 파일 (전체 구조 포함)
- CSV 파일 (분석용 테이블 형식)
- 파일명: `summaries_batch_{타임스탬프}.json/csv`

In [ ]:
# ========================================
# 결과 저장
# ========================================

from datetime import datetime
import pandas as pd

if len(all_results) == 0:
    print("⚠️ 저장할 결과가 없습니다.")
    print("   먼저 섹션 6을 실행하여 요약문을 생성하세요.")
else:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    if MODE == "single":
        # -----------------------------------------
        # 단일 매장: JSON 파일로 저장
        # -----------------------------------------
        result_json = all_results[0]
        filename = f"summary_{result_json.get('shop_name')}_{timestamp}.json"
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(result_json, f, ensure_ascii=False, indent=2)
        
        print(f"✅ JSON 저장 완료: {filename}")
        print(f"   - 매장명: {result_json.get('shop_name')}")
        print(f"   - 제목: {result_json.get('title')}")
        print(f"   - 요약문: {len(result_json.get('summaries', []))}개")
    
    elif MODE == "multi":
        # -----------------------------------------
        # 다중 매장: JSON + CSV 파일로 저장
        # -----------------------------------------
        
        # 1. JSON 파일 (전체 구조 그대로)
        json_filename = f"summaries_batch_{timestamp}.json"
        with open(json_filename, 'w', encoding='utf-8') as f:
            json.dump(all_results, f, ensure_ascii=False, indent=2)
        
        print(f"✅ JSON 저장 완료: {json_filename}")
        print(f"   - 총 매장 수: {len(all_results)}개")
        
        # 2. CSV 파일 (분석용)
        csv_data = []
        for result in all_results:
            if result.get("status") != "error":
                csv_data.append({
                    "shop_seq": result.get("shop_seq"),
                    "shop_name": result.get("shop_name"),
                    "title": result.get("title"),
                    "summary_1": result.get("summaries", [])[0] if len(result.get("summaries", [])) > 0 else "",
                    "summary_2": result.get("summaries", [])[1] if len(result.get("summaries", [])) > 1 else "",
                    "summary_3": result.get("summaries", [])[2] if len(result.get("summaries", [])) > 2 else "",
                    "status": "success"
                })
            else:
                csv_data.append({
                    "shop_seq": result.get("shop_seq"),
                    "shop_name": result.get("shop_name"),
                    "title": "",
                    "summary_1": "",
                    "summary_2": "",
                    "summary_3": "",
                    "status": "error",
                    "error": result.get("error", "")
                })
        
        df = pd.DataFrame(csv_data)
        csv_filename = f"summaries_batch_{timestamp}.csv"
        df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        
        print(f"✅ CSV 저장 완료: {csv_filename}")
        
        # 성공/실패 통계
        success_count = len(df[df['status'] == 'success'])
        error_count = len(df[df['status'] == 'error'])
        
        print(f"\n📊 통계:")
        print(f"   - 성공: {success_count}개")
        if error_count > 0:
            print(f"   - 실패: {error_count}개")
        
        # DataFrame 미리보기
        print(f"\n📋 결과 미리보기:")
        print(df[['shop_name', 'title', 'status']].to_string(index=False))